In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import multiprocessing
from transformers import *
from datasets import load_dataset
from PIL import Image
from torchinfo import summary
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import time

torch.cuda.empty_cache()
device="cuda" if torch.cuda.is_available() else "cpu"

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.0.1+cu117)
    Python  3.9.18 (you have 3.9.17)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


### Importing Data

In [2]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [3]:
model_name = "microsoft/swin-base-patch4-window7-224-in22k"
batch_size = 16
cpu_count=multiprocessing.cpu_count()


image_processor = AutoImageProcessor.from_pretrained(model_name)


train_ds= load_dataset('./chest_xray/data')
train_ds = train_ds["train"].train_test_split(test_size=0.25) 

labels = train_ds["train"].features["label"].names
dataset = train_ds.with_transform(transform)


train_dataset_loader = torch.utils.data.DataLoader(dataset["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = torch.utils.data.DataLoader(dataset["test"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)


loading configuration file preprocessor_config.json from cache at /home/moose/.cache/huggingface/hub/models--microsoft--swin-base-patch4-window7-224-in22k/snapshots/68dc76680a5bf3bdf670669f3025dc9be2e30781/preprocessor_config.json
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Resolving data files:   0%|          | 0/5840 [00:00<?, ?it/s]

### Loading Model

In [4]:
model = SwinForImageClassification.from_pretrained(model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,
)


model = model.to(device)

loading configuration file config.json from cache at /home/moose/.cache/huggingface/hub/models--microsoft--swin-base-patch4-window7-224-in22k/snapshots/68dc76680a5bf3bdf670669f3025dc9be2e30781/config.json
Model config SwinConfig {
  "_name_or_path": "microsoft/swin-base-patch4-window7-224-in22k",
  "architectures": [
    "SwinForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    18,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "NORMAL",
    "1": "PNEUMONIA"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "NORMAL": "0",
    "PNEUMONIA": "1"
  },
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "patch_

### Optimizer and accuracy functions

In [5]:
#Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

log_dir = "./swin_base4_224_tensorboard/"
writer = SummaryWriter(log_dir=log_dir)

def accuracy_fn(logits,true):
    return torch.eq(torch.argmax(torch.softmax(logits,dim=1),dim=1).squeeze(),true).sum().item()/len(logits)
    

In [6]:
epochs=10

train_accuracy=[]
test_accuracy=[]
train_loss=[]
test_loss=[]

for i in tqdm(range(epochs)):
    print("Training:")
    model.train()
    
    #Defining accuracy and loss for train and test data
    temp_train_accuracy=[]
    temp_test_accuracy=[]
    temp_train_loss=[]
    temp_test_loss=[]

    net_train_accuracy=0
    net_test_accuracy=0
    net_train_loss=0
    net_test_loss=0
    
    with tqdm(total=len(train_dataset_loader)) as pbar:
        for batch in train_dataset_loader:
                x=batch["pixel_values"].to(device)
                y=batch["labels"].to(device)
    
                #Calculating model output
                result=model(pixel_values=x,labels=y)
                logits=result.logits
    
                #Reseting any old gradient values
                optimizer.zero_grad()
                #loss=loss_fn(logits.squeeze(),y.type(torch.float32))
                loss=result.loss

            
                #Track of metrics        
                accuracy_train=accuracy_fn(logits,y)
                temp_train_accuracy.append(accuracy_train)
                temp_train_loss.append(loss.item())
    
                #Back Propogation
                loss.backward()
            
                #Update Parameters
                optimizer.step()
            
                #Progress Bar Update
                pbar.update(1)
        pbar.close()
    #Tensorboard & Metrics for the dataset
    net_train_accuracy=sum(temp_train_accuracy)/len(temp_train_accuracy)
    net_train_loss=sum(temp_train_loss)/len(temp_train_loss)
    train_accuracy.append(net_train_accuracy)
    train_loss.append(net_train_loss)
    writer.add_scalar("Train Accuracy",net_train_accuracy,i)
    writer.add_scalar("Train Loss",net_train_loss,i)

    #Evaluation
    print("Testing:")
    model.eval()

    with tqdm(total=len(valid_dataset_loader)) as pbar2:
        for batch in valid_dataset_loader:
            x=batch["pixel_values"].to(device)
            y=batch["labels"].to(device)
            
            #Setting inference mode
            with torch.inference_mode():
                result=model(pixel_values=x,labels=y)
                logits=result.logits
                #loss=loss_fn(logits.squeeze(),y.type(torch.float32))
                loss=result.loss

                #Track of metrics
                accuracy_test=accuracy_fn(logits,y)
                temp_test_accuracy.append(accuracy_test)
                temp_test_loss.append(loss.item())

                #Progress Bar Update
                pbar2.update(1)
        pbar2.close()

    #Tensorboard & Metrics for the dataset
    net_test_accuracy=sum(temp_test_accuracy)/len(temp_test_accuracy)
    net_test_loss=sum(temp_test_loss)/len(temp_test_loss)
    test_accuracy.append(net_test_accuracy)
    test_loss.append(net_test_loss)
    writer.add_scalar("Test Accuracy",net_test_accuracy,i)
    writer.add_scalar("Test Loss",net_test_loss,i)

    #Saving the model
    model.save_pretrained(f"./swin-base4-224/checkpoint-{i+1}")
    image_processor.save_pretrained(f"./swin-base4-224/checkpoint-{i+1}")

    print(f"Epoch {i+1}:\nTrain Accuracy: {net_train_accuracy}  Train Loss: {net_train_loss}  Test Accuracy: {net_test_accuracy}  Test Loss: {net_test_loss}")
    print("\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Training:


  0%|          | 0/274 [00:00<?, ?it/s]

Testing:


  0%|          | 0/92 [00:00<?, ?it/s]

...

  0%|          | 0/274 [00:00<?, ?it/s]

Testing:


  0%|          | 0/92 [00:00<?, ?it/s]

Configuration saved in ./swin-base4-224/checkpoint-10/config.json
Model weights saved in ./swin-base4-224/checkpoint-10/pytorch_model.bin
Image processor saved in ./swin-base4-224/checkpoint-10/preprocessor_config.json


Epoch 10:
Train Accuracy: 0.9978710462287105  Train Loss: 0.0067393360615200315  Test Accuracy: 0.9769021739130435  Test Loss: 0.10731639343818307


